# ML Iteration 2: Removing the draws from the winner column

In [1]:
# Perform initial import of the needed libraries

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import sklearn
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from IPython.display import display

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

In [52]:
# Read the CSV and perform basic data cleaning
file_path = Path('../Resources/fights_fighters_records_v2.csv')
fnf_df = pd.read_csv(file_path)[:-2]

In [53]:
fnf_df.columns

Index(['fight_id', 'winner', 'title_bout', 'weight_class', 'no_of_rounds',
       'r_fighter', 'r_fighter_id', 'r_fighter_stance', 'r_fighter_age',
       'b_fighter', 'b_fighter_id', 'b_fighter_stance', 'b_fighter_age',
       'date', 'referee', 'referee_id', 'b_fighter_height', 'r_fighter_height',
       'b_fighter_reach', 'r_fighter_reach', 'b_fighter_weight',
       'r_fighter_weight', 'records_id', 'fight_id-2', 'b_current_lose_streak',
       'b_current_win_streak', 'b_draw', 'b_longest_win_streak', 'b_losses',
       'b_total_title_bouts', 'b_win_by_decision_majority',
       'b_win_by_decision_split', 'b_win_by_decision_unanimous',
       'b_win_by_ko_tko', 'b_win_by_submission',
       'b_win_by_tko_doctor_stoppage', 'b_wins', 'b_total_rounds_fought',
       'b_total_time_fought_seconds', 'r_current_lose_streak',
       'r_current_win_streak', 'r_draw', 'r_longest_win_streak', 'r_losses',
       'r_total_rounds_fought', 'r_total_time_fought_seconds',
       'r_total_title_bout

In [54]:
columns = (['winner', 'title_bout', 'weight_class', 'no_of_rounds', 'r_fighter_stance', 'b_fighter_stance', 'b_fighter_height', 'r_fighter_height', 'b_fighter_reach',
       'r_fighter_reach', 'b_fighter_weight', 'r_fighter_weight', 'b_fighter_age', 'r_fighter_age'])

target = ["winner"]

In [55]:
fnf_df = fnf_df.loc[:, columns].copy()

In [56]:
fnf_df = fnf_df[fnf_df.weight_class != "Women's Bantamweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Featherweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Flyweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Strawweight"]
fnf_df = fnf_df[fnf_df.winner != "Draw"]
fnf_df.tail(50)

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
5079,Blue,FALSE,Welterweight,3,Orthodox,Orthodox,177.80,180.34,180.34,193.04,170.0,170.0,27.0,31.0
5080,Red,FALSE,Lightweight,3,Orthodox,Orthodox,185.42,187.96,190.50,185.42,155.0,155.0,26.0,25.0
5081,Red,FALSE,Heavyweight,3,Orthodox,Switch,187.96,195.58,187.96,203.20,253.0,265.0,36.0,30.0
5082,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,190.50,177.80,145.0,145.0,28.0,35.0
5083,Blue,FALSE,Lightweight,5,Orthodox,Orthodox,185.42,177.80,185.42,177.80,155.0,155.0,36.0,32.0
5084,Blue,FALSE,Bantamweight,3,Southpaw,Orthodox,180.34,165.10,170.18,165.10,135.0,135.0,30.0,34.0
5085,Red,FALSE,Middleweight,3,Southpaw,Orthodox,185.42,185.42,190.50,195.58,185.0,185.0,30.0,35.0
5086,Red,FALSE,Heavyweight,3,Southpaw,Orthodox,190.50,195.58,198.12,195.58,260.0,250.0,24.0,35.0
5087,Red,FALSE,Welterweight,3,Orthodox,Southpaw,177.80,185.42,177.80,187.96,170.0,170.0,29.0,37.0
5088,Blue,FALSE,Bantamweight,3,Orthodox,Orthodox,170.18,172.72,177.80,172.72,135.0,135.0,28.0,31.0


In [57]:
# for row in fnf_df.weight_class:
#     if fnf_df["weight_class"] == "Women's Bantamweight":
#         print ("found one")a

In [58]:
fnf_df.isnull().sum()

winner                0
title_bout            0
weight_class          0
no_of_rounds          0
r_fighter_stance     89
b_fighter_stance    114
b_fighter_height      8
r_fighter_height      4
b_fighter_reach     651
r_fighter_reach     305
b_fighter_weight      6
r_fighter_weight      3
b_fighter_age       170
r_fighter_age        63
dtype: int64

In [59]:
fnf_df['r_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['b_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['r_fighter_height'].fillna(value=fnf_df['r_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_height'].fillna(value=fnf_df['b_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_reach'].fillna(value=fnf_df['b_fighter_reach'].mean(), inplace = True)
fnf_df['r_fighter_reach'].fillna(value=fnf_df['r_fighter_reach'].mean(), inplace = True)
fnf_df['b_fighter_weight'].fillna(value=fnf_df['b_fighter_weight'].mean(), inplace = True)
fnf_df['r_fighter_weight'].fillna(value=fnf_df['r_fighter_weight'].mean(), inplace = True)
fnf_df['b_fighter_age'].fillna(value=fnf_df['b_fighter_age'].mean(), inplace = True)
fnf_df['r_fighter_age'].fillna(value=fnf_df['r_fighter_age'].mean(), inplace = True)

In [60]:
fnf_df.isnull().sum()

winner              0
title_bout          0
weight_class        0
no_of_rounds        0
r_fighter_stance    0
b_fighter_stance    0
b_fighter_height    0
r_fighter_height    0
b_fighter_reach     0
r_fighter_reach     0
b_fighter_weight    0
r_fighter_weight    0
b_fighter_age       0
r_fighter_age       0
dtype: int64

In [61]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0
5134,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0
5135,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0
5136,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0


In [92]:
# Reset the indexes and remove the original index
fnf_df = fnf_df.reset_index(drop=True)
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0
4746,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0
4747,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0
4748,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0


In [63]:
fnf_df.dtypes

winner               object
title_bout           object
weight_class         object
no_of_rounds          int64
r_fighter_stance     object
b_fighter_stance     object
b_fighter_height    float64
r_fighter_height    float64
b_fighter_reach     float64
r_fighter_reach     float64
b_fighter_weight    float64
r_fighter_weight    float64
b_fighter_age       float64
r_fighter_age       float64
dtype: object

In [64]:
# Generate our categorical variable lists
app_cat = fnf_df.dtypes[fnf_df.dtypes == "object"].index.tolist()

In [65]:
app_cat = [
 'title_bout',
 'weight_class',
 'r_fighter_stance',
 'b_fighter_stance']

In [66]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fnf_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [67]:
pd.options.display.max_columns = None
display(encode_df)

,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4746,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4747,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4748,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [68]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0
4746,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0
4747,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0
4748,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0


In [69]:
encode_df.columns

Index(['title_bout_FALSE', 'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'],
      dtype='object')

In [70]:
fnf_df_num = pd.concat([fnf_df,encode_df],axis=1)

In [71]:
fnf_df_num

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4746,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4747,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4748,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [72]:
fnf_df_num_2 = fnf_df_num.copy()

In [73]:
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Red ", value = 1.0)
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Blue", value = 2.0)
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Draw", value = 3.0)

fnf_df_num_2.head(40)

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1.0,TRUE,Catch Weight,1,Southpaw,Orthodox,195.580000,185.420000,184.507236,184.820571,216.0000,175.000000,34.000000,26.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,FALSE,Open Weight,1,Southpaw,Orthodox,185.420000,185.420000,182.880000,184.820571,205.0000,175.000000,29.000000,26.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,FALSE,Open Weight,1,Southpaw,Orthodox,185.420000,185.420000,184.507236,184.820571,196.0000,175.000000,30.000000,26.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,FALSE,Open Weight,1,Southpaw,unlisted,187.960000,180.340000,184.507236,184.820571,185.0000,190.000000,29.186463,24.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,193.040000,195.580000,184.507236,184.820571,275.0000,216.000000,29.186463,34.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,182.880000,195.580000,184.507236,184.820571,430.0000,216.000000,24.000000,34.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,187.960000,185.420000,184.507236,182.880000,225.0000,205.000000,30.000000,29.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,195.580000,193.040000,184.507236,184.820571,250.0000,275.000000,29.186463,29.448261,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,FALSE,Open Weight,1,Southpaw,Southpaw,190.500000,185.420000,184.507236,184.820571,260.0000,175.000000,29.186463,27.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,FALSE,Open Weight,1,Southpaw,Southpaw,180.340000,185.420000,184.507236,184.820571,190.0000,175.000000,24.000000,27.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [74]:
fnf_df_num_2.drop(['title_bout', 'weight_class', 'r_fighter_stance', 'b_fighter_stance'], axis = 1 , inplace = True)
fnf_df_num_2

,winner,no_of_rounds,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1.0,1,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,1,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,2.0,3,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4746,1.0,3,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4747,1.0,5,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4748,2.0,3,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [75]:
fnf_df_num_2.dtypes

winner                            float64
no_of_rounds                        int64
b_fighter_height                  float64
r_fighter_height                  float64
b_fighter_reach                   float64
r_fighter_reach                   float64
b_fighter_weight                  float64
r_fighter_weight                  float64
b_fighter_age                     float64
r_fighter_age                     float64
title_bout_FALSE                  float64
title_bout_TRUE                   float64
weight_class_Bantamweight         float64
weight_class_Catch Weight         float64
weight_class_Featherweight        float64
weight_class_Flyweight            float64
weight_class_Heavyweight          float64
weight_class_Light Heavyweight    float64
weight_class_Lightweight          float64
weight_class_Middleweight         float64
weight_class_Open Weight          float64
weight_class_Welterweight         float64
r_fighter_stance_Open Stance      float64
r_fighter_stance_Orthodox         

In [76]:
# Preprocess data, split the data into Training and Testing
    # Create feature set 
X = fnf_df_num_2.copy()
X = X.drop(columns='winner', axis=1)
X

,no_of_rounds,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1,195.58,185.42,184.507236,184.820571,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,185.42,185.42,182.880000,184.820571,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,185.42,185.42,184.507236,184.820571,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,187.96,180.34,184.507236,184.820571,185.0,190.0,29.186463,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,193.04,195.58,184.507236,184.820571,275.0,216.0,29.186463,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,3,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4746,3,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4747,5,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4748,3,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [77]:
# Create target
y = fnf_df_num_2["winner"]
y

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4745    2.0
4746    1.0
4747    1.0
4748    2.0
4749    2.0
Name: winner, Length: 4750, dtype: float64

In [78]:
# Check the balance of our target values
y.value_counts()

1.0    3288
2.0    1462
Name: winner, dtype: int64

In [79]:
# Split into Train and Test sets. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [80]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3800, 33)
(950, 33)
(3800,)
(950,)


In [81]:
# Create a StandardScaler instance. 
scaler = StandardScaler()

In [82]:
# Fit the Standard Scaler with the training data. 
X_scaler = scaler.fit(X_train)

In [83]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [84]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [85]:
y_pred = brf.predict(X_test)

In [86]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6342315369261478

In [87]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[402, 266],
       [ 94, 188]], dtype=int64)

In [88]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        1.0       0.81      0.60      0.67      0.69      0.63      0.40       668
        2.0       0.41      0.67      0.60      0.51      0.63      0.40       282

avg / total       0.69      0.62      0.65      0.64      0.63      0.40       950



In [89]:
X.columns

Index(['no_of_rounds', 'b_fighter_height', 'r_fighter_height',
       'b_fighter_reach', 'r_fighter_reach', 'b_fighter_weight',
       'r_fighter_weight', 'b_fighter_age', 'r_fighter_age',
       'title_bout_FALSE', 'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'],
      dtype='object')

In [90]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X_train.columns), reverse=True)

[(0.14547501964526383, 'r_fighter_age'),
 (0.13286163325543782, 'b_fighter_age'),
 (0.1280063991173726, 'b_fighter_reach'),
 (0.10731086061361664, 'r_fighter_reach'),
 (0.09610023142464977, 'b_fighter_height'),
 (0.0933962401265471, 'r_fighter_height'),
 (0.05710931833967072, 'b_fighter_weight'),
 (0.05420734100262785, 'r_fighter_weight'),
 (0.02279306049160456, 'no_of_rounds'),
 (0.018303950477844, 'r_fighter_stance_Orthodox'),
 (0.01603064311104294, 'b_fighter_stance_Orthodox'),
 (0.016002252915659196, 'r_fighter_stance_Southpaw'),
 (0.015727393808132477, 'b_fighter_stance_Southpaw'),
 (0.01008993272847882, 'weight_class_Lightweight'),
 (0.009834455471287532, 'weight_class_Welterweight'),
 (0.008053907473563986, 'weight_class_Middleweight'),
 (0.007393323367405244, 'weight_class_Featherweight'),
 (0.007061792509676853, 'title_bout_TRUE '),
 (0.006398316295721646, 'b_fighter_stance_Switch'),
 (0.00635487687112284, 'title_bout_FALSE'),
 (0.006130322302045634, 'r_fighter_stance_Switch')